# Box touches box: vertex-edge

In [1]:
import numpy as np

box_A_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="Box_A">
    <link name="box_A">
      <collision name="collision">
        <geometry>
          <box>
            <size>2 2 2</size>
          </box>
        </geometry>
      </collision>
      <visual name="visual">
        <geometry>
          <box>
            <size>2 2 2</size>
          </box>
        </geometry>
        <material>
          <diffuse>1.0 1.0 1.0 1</diffuse>
        </material>
      </visual>
    </link>
  </model>
</sdf>
"""

box_B_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="Box_B">
    <link name="box_B">
      <collision name="collision">
        <geometry>
          <box>
            <size>2 2 2</size>
          </box>
        </geometry>
      </collision>
      <visual name="visual">
        <geometry>
          <box>
            <size>2 2 2</size>
          </box>
        </geometry>
        <material>
         <diffuse>0.9 0.8 0.7 1</diffuse>
        </material>
      </visual>
    </link>
  </model>
</sdf>
"""

from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0)
parser = Parser(plant)

parser.AddModelsFromString(box_A_sdf, "sdf")
parser.AddModelsFromString(box_B_sdf, "sdf")
plant.Finalize()

X_WA= RigidTransform(rpy=RollPitchYaw(np.pi / 4, np.pi / 12, np.pi / 4),
                      p=np.zeros(3))
plant.SetDefaultFreeBodyPose(plant.GetBodyByName("box_A"), X_WA)

p_ACa = [0, -1., -1.]
p_WCa = X_WA.multiply(p_BoQ_B=p_ACa)
p_WBo = p_WCa - [1., 1., 1.]

X_WB = RigidTransform(p = p_WBo)
plant.SetDefaultFreeBodyPose(plant.GetBodyByName("box_B"), X_WB)    

AddDefaultVisualization(builder=builder)

diagram = builder.Build()


from pydrake.systems.analysis import Simulator
simulator = Simulator(diagram)
simulator.Initialize()
simulator.AdvanceTo(0)

context = diagram.CreateDefaultContext()
scene_graph_context = scene_graph.GetMyMutableContextFromRoot(context)
query_object = scene_graph.get_query_output_port().Eval(scene_graph_context)
signed_distance_pairs = query_object.ComputeSignedDistancePairwiseClosestPoints()

for pair in signed_distance_pairs:
    print(f"id_A = {pair.id_A}")
    print(f"id_B = {pair.id_B}")
    print(f"p_ACa = {pair.p_ACa.transpose()}")
    print(f"p_BCb = {pair.p_BCb.transpose()}")
    print(f"distance = {pair.distance}")
    print(f"nhat_BA_W = {pair.nhat_BA_W.transpose()}")
    print()

INFO:drake:Meshcat listening for connections at http://localhost:7000


id_A = <GeometryId value=21>
id_B = <GeometryId value=27>
p_ACa = [ 2.77555756e-16 -1.00000000e+00 -1.00000000e+00]
p_BCb = [1. 1. 1.]
distance = -1.9229626863835638e-16
nhat_BA_W = [-0.          0.35434932  0.93511313]



In [2]:
Na_A = [1, 0, 0]
Na_W = X_WA.rotation().multiply(v_B = Na_A)
print(f"Na_A = {Na_A}")
print(f"Na_W = {Na_W}")

Nb_B = [1, 0, 0]
Nb_W = X_WB.rotation().multiply(v_B = Nb_B)
print(f"Nb_B = {Nb_B}")
print(f"Nb_W = {Nb_W}")

Nb_W_x_Na_W = np.cross(Nb_W, Na_W)
print(f"Nb_W_x_Na_W = {Nb_W_x_Na_W}")

normalized_Nb_x_Na_W = Nb_W_x_Na_W / np.linalg.norm(Nb_W_x_Na_W)
print(f"normalized_Nb_x_Na_W = {normalized_Nb_x_Na_W}")

Na_A = [1, 0, 0]
Na_W = [ 0.6830127   0.6830127  -0.25881905]
Nb_B = [1, 0, 0]
Nb_W = [1. 0. 0.]
Nb_W_x_Na_W = [-0.          0.25881905  0.6830127 ]
normalized_Nb_x_Na_W = [-0.          0.35434932  0.93511313]
